In [446]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from itertools import product
from torch_geometric.data import Data
import numpy as np
import torch
from torch_geometric.nn import GATConv,Linear, SAGEConv, GCNConv

In [447]:
hierarchie_information = pd.read_csv("Cell_listTue Jul  9 10_41_58 2024.csv")
hierarchie_information = hierarchie_information.rename(columns={"Model ID": "SANGER_MODEL_ID"})
hierarchie_information_red = hierarchie_information.loc[:, ["SANGER_MODEL_ID", "Tissue sub-type"]]

In [448]:
hierarchie_information_red["Tissue sub-type"].unique().shape

(56,)

In [449]:
perfect_features = ['FIGN', 'FAR2P2', 'UBAC2-AS1', 'PKM', 'LINC01752', 'FOXCUT',
       'COL9A3', 'SPP1', 'CEP170P1', 'NF2', 'INPP1', 'ARL9', 'BTBD19',
       'LYPD1', 'MYH14', 'ELOVL7', 'ROMO1', 'HSPE1P21', 'RPS26P21',
       'FUT8', 'TMEM127', 'POU5F1P5', 'MANCR', 'GLO1', 'VCP', 'LPGAT1',
       'C4BPB', 'ADGRF4', 'CCDC57', 'KLK8', 'LINC02551', 'NGEF',
       'ANKDD1B', 'SNHG30', 'LINC01133', 'FAM189A2', 'HMMR', 'KRTAP3-1',
       'TIAM1', 'SHKBP1', 'FASTKD3', 'CARD14', 'SPATS2L', 'TMED7-TICAM2',
       'PABPC1P3', 'MTND6P4', 'EPS8L1', 'MIR99AHG', 'PCSK6-AS1', 'IRF6',
       'SNORD124', 'A2M-AS1', 'SMOC1', 'SLC22A3', 'IGFBP6', 'ANXA2R',
       'PPFIA3', 'CD9', 'GAS2L1', 'HEIH', 'GGACT', 'ANKRA2', 'NUP210',
       'FAM43B', 'MIR574', 'KRT23', 'CTU1', 'PELO', 'ERICH5', 'LRRC2',
       'ZNF17', 'LPP-AS2', 'HLA-F-AS1', 'RNU6-37P', 'MTMR11', 'TAGLN',
       'AKAP12', 'PROM2', 'LYPLAL1-DT', 'PCDHB17P', 'KCNH3', 'CCDC61',
       'RPS2P55', 'SCARNA8', 'LINC01694', 'SNHG25', 'INHA', 'CHMP6',
       'SLC44A4', 'PIGC', 'ERCC6', 'MAGEH1', 'OR7E91P', 'RAPGEF4',
       'RPS20', 'NECTIN3-AS1', 'LINC00622', 'RCN1P2', 'IPO4', 'VSIG10L',
       'IMPA1P1', 'KLK7', 'IGFBP5', 'GJC2', 'NFRKB', 'RPL35P6', 'ADAM33',
       'GDI2', 'BEND7', 'USPL1', 'FLRT3', 'RBM23', 'KLK14', 'AGAP14P',
       'LINC02331', 'M1AP', 'UCN2', 'OSBPL9', 'SEMA4F', 'ERMP1', 'OPLAH',
       'TMEM87A', 'ZNF674-AS1', 'INSIG2', 'MIR3192', 'NELFA', 'RPSAP58',
       'WBP4', 'TCIRG1', 'AMZ1', 'FGFBP1', 'CDC27P1', 'COLGALT2', 'TGM2',
       'PEDS1', 'MMP10', 'RPL7AP11', 'MFSD14A', 'TMEM253', 'DNAH5',
       'TMLHE', 'HERC5', 'CNPY2', 'SNAPC2', 'GJB7', 'OCRL', 'MGLL',
       'ANO1', 'CNOT3', 'SLC5A6', 'NAT1', 'ANKRD40', 'PLEKHG4B', 'FLG',
       'LINC02340', 'ABHD11', 'LINC00659', 'RN7SL552P', 'IHO1', 'AP1M2',
       'CCNB1IP1', 'AK4', 'SULT1C4', 'IGFL2-AS1', 'SVIL-AS1', 'PTPA',
       'PLXDC1', 'SLC30A3', 'SH3BP2', 'KCNK2', 'MTCO3P12', 'MIR6835',
       'MRPL37', 'MAGEF1', 'PRR34-AS1', 'HDLBP', 'SDR42E1', 'CASP7',
       'NKILA', 'AFF1', 'NUPR1', 'CUBN', 'TGFBR3', 'PPP1R10', 'MIR181B2',
       'LIF', 'IFRD1', 'TLL2', 'ERICH6-AS1', 'RNU6-589P', 'A1BG-AS1',
       'MSX2', 'MFSD3', 'LINC02102', 'LDHAL6FP', 'PTGR1', 'CDYL2', 'PEX3',
       'EEF1A1P11', 'LSMEM2', 'LINC01040', 'ZNF776', 'SLC44A2', 'CCDC9',
       'EXOSC2', 'ATG101', 'MYD88', 'SNORA79B', 'SLC3A2', 'CERNA1',
       'LYPD3', 'USP2', 'RPS20P2', 'VNN1', 'KREMEN2', 'HTR1D', 'TBC1D3C',
       'RPS6KA2', 'CHKB-CPT1B', 'RNU6-469P', 'CXADR', 'BAZ1A',
       'LINC00960', 'FTH1P23', 'RNASEK-C17orf49', 'VILL', 'BNIP3P5',
       'PARP3', 'COBL', 'TGFB3', 'CDO1', 'CT45A11P', 'LASP1', 'HYI-AS1',
       'NRBP2', 'CYB561A3', 'C1QTNF12', 'IQUB', 'MIR6499', 'TAFA3',
       'TRPV6', 'CCL20', 'SLC16A3', 'RASA1', 'POMT1', 'RBM17', 'CYP4X1',
       'SPDEF', 'VEGFC', 'APOBEC3A', 'MIR4754', 'METTL15P1', 'PNKP',
       'UBE2L3', 'ARSK', 'BACH1-AS1', 'TRPM4', 'KCTD14', 'NNT-AS1',
       'B3GALNT1', 'PRSS35', 'UGT1A3', 'MIR6826', 'PRSS41', 'MT-CO3',
       'SIRT2', 'AGAP12P', 'SNX18P9', 'YBX1P1', 'ADRA1B', 'ACTL11P',
       'MINDY3', 'CEP250', 'STAG3L5P', 'TGFA-IT1', 'IER5', 'UNC5CL',
       'HNRNPA3P12', 'FGB', 'SOD3', 'OBSL1', 'STX10', 'PDRG1', 'SPATA17',
       'PCED1B', 'EFNB2', 'CHCHD2P2', 'GYG1P1', 'CD109', 'EEF2K',
       'ADAMTSL4', 'PERM1', 'CDC20B', 'RGS5', 'WNT2', 'PCBP2', 'HOMEZ',
       'RPS12P27', 'ZNF615', 'BMS1P11', 'ZNF577', 'CFAP298', 'NRSN2',
       'TMEM139', 'SEPTIN7P3', 'RNU11-2P', 'RPL31P59', 'RASSF3', 'EFHD1',
       'ORAI3', 'RNU6-100P', 'ZNF662', 'VAMP8', 'HEPHL1', 'FSTL5',
       'FAM3D', 'HS3ST1', 'NUMB', 'MYLK4', 'CLDN16', 'LINC00460', 'PROM1',
       'PRRG2', 'CCDC6', 'GSC', 'CCDC157', 'GABARAPL1', 'RN7SKP160',
       'PGLYRP1', 'RPS5', 'BCAN', 'FAM107A', 'ADAMTS13', 'OR6A2',
       'ARPC1B', 'D2HGDH', 'JAZF1', 'KRTAP2-1', 'ARHGAP23', 'DEFB1',
       'ZNF584', 'LRP1B', 'POTEI', 'NLRP1', 'RNY1', 'C6orf15', 'B4GAT1',
       'SCARF1', 'LINC00643', 'CSGALNACT1', 'MIR1262', 'RNU6-377P',
       'RTN4RL1', 'JOSD2', 'ATXN7L3B', 'KCP', 'MAP2K1', 'C2orf88',
       'LAPTM4A', 'SSTR5-AS1', 'DAPK3', 'ZNF75D', 'PIGV', 'SAV1', 'PCGF2',
       'TOB2P1', 'DNAJB12', 'AK2P2', 'YWHABP2', 'CPT1B', 'PLK3', 'PODXL',
       'RPSAP18', 'PRPF6', 'VPS26BP1', 'CD47', 'PRR36', 'HERC2P3',
       'TMEM134', 'ATP8B1', 'WNT4', 'ERV3-1', 'PSMD10P2', 'SCRN3',
       'CDK10', 'CTSV', 'RNA5SP440', 'EIF1AD']
features = list(map(lambda s: s+ "_exp", perfect_features))

## Graph Strcuture

In [450]:
def get_transformed_df_in_model_gene_value_format(rna_seq_df):
    rna_seq_df_transposed = rna_seq_df.transpose().iloc[0:, :]
    new_header = rna_seq_df_transposed.iloc[0, :] 
    rna_seq_df_transposed = rna_seq_df_transposed.iloc[1:, :] 
    rna_seq_df_transposed.columns = new_header
    rna_seq_df_transposed = rna_seq_df_transposed.iloc[:, [0, *list(range(5, new_header.shape[0]))]]
    rna_seq_df_transposed = rna_seq_df_transposed.rename(columns={rna_seq_df_transposed.columns[0]: "SANGER_MODEL_ID"})
    rna_seq_df_transposed_stacked = rna_seq_df_transposed.set_index('SANGER_MODEL_ID').stack().reset_index(name='ExpressionValue')
    rna_seq_df_transposed_stacked = rna_seq_df_transposed_stacked.rename(columns={1:'symbol'})
    return rna_seq_df_transposed_stacked

## Gene expression information

In [451]:
rna_seq_df = pd.read_csv("rnaseq_tpm_20220624.csv",index_col=0, header = None,low_memory=False)
transformed_seq_data = get_transformed_df_in_model_gene_value_format(rna_seq_df)
model_expression_data = pd.pivot_table(transformed_seq_data,  values='ExpressionValue', index=['SANGER_MODEL_ID'],
                       columns=['symbol'], aggfunc="sum", fill_value=0.0).reset_index()
filtered_model_expression_data = model_expression_data.loc[:, ["SANGER_MODEL_ID", *perfect_features]]

## Target information

In [452]:
targets = pd.read_csv("target.csv",sep=";")
drug_targets = targets[targets["DRUG_ID"] == 1803]
models = pd.read_csv("model_list_20240110.csv", sep = ",")

In [453]:
drug_targets

,DATASET,NLME_RESULT_ID,NLME_CURVE_ID,COSMIC_ID,CELL_LINE_NAME,SANGER_MODEL_ID,TCGA_DESC,DRUG_ID,DRUG_NAME,PUTATIVE_TARGET,PATHWAY_NAME,COMPANY_ID,WEBRELEASE,MIN_CONC,MAX_CONC,LN_IC50,AUC,RMSE,Z_SCORE
144335,GDSC2,401,18945693,683667,PFSK-1,SIDM01132,MB,1803,Acetalax,NaN,Unclassified,1043,Y,0.030016,30.0,5.078148,0.961643,0.016681,0.063499
144336,GDSC2,401,18945958,684052,A673,SIDM00848,UNCLASSIFIED,1803,Acetalax,NaN,Unclassified,1043,Y,0.030016,30.0,2.849687,0.839819,0.094560,-1.136831
144337,GDSC2,401,18946245,684057,ES5,SIDM00263,UNCLASSIFIED,1803,Acetalax,NaN,Unclassified,1043,Y,0.030016,30.0,5.814401,0.980146,0.077549,0.460072
144338,GDSC2,401,18946497,684059,ES7,SIDM00269,UNCLASSIFIED,1803,Acetalax,NaN,Unclassified,1043,Y,0.030016,30.0,3.390164,0.883963,0.132560,-0.845710
144339,GDSC2,401,18946779,684062,EW-11,SIDM00203,UNCLASSIFIED,1803,Acetalax,NaN,Unclassified,1043,Y,0.030016,30.0,6.030742,0.971616,0.093845,0.576601
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145047,GDSC2,401,19187832,1660034,SNU-407,SIDM00214,COREAD,1803,Acetalax,NaN,Unclassified,1043,Y,0.030016,30.0,4.973348,0.938441,0.048938,0.007050
145048,GDSC2,401,19188102,1660035,SNU-61,SIDM00194,COREAD,1803,Acetalax,NaN,Unclassified,1043,Y,0.030016,30.0,3.812519,0.885263,0.042282,-0.618215
145049,GDSC2,401,19188370,1660036,SNU-81,SIDM00193,COREAD,1803,Acetalax,NaN,Unclassified,1043,Y,0.030016,30.0,2.698528,0.834159,0.078634,-1.218251
145050,GDSC2,401,19188621,1674021,SNU-C5,SIDM00498,COREAD,1803,Acetalax,NaN,Unclassified,1043,Y,0.030016,30.0,7.079313,0.978743,0.067601,1.141399


## Merge target and gene expression

In [454]:
merged_target_genes = pd.merge(drug_targets, filtered_model_expression_data, on = "SANGER_MODEL_ID")

## Train-test-split

In [479]:
from sklearn.model_selection import train_test_split

train_idx, test_idx = train_test_split(np.arange(merged_target_genes.shape[0]),random_state=42)
X_train = merged_target_genes[perfect_features].iloc[train_idx].values
X_test = merged_target_genes[perfect_features].iloc[test_idx].values
y_train = merged_target_genes["LN_IC50"].iloc[train_idx].values
y_test = merged_target_genes["LN_IC50"].iloc[test_idx].values

## Benchmark

In [480]:
from xgboost import XGBRegressor
lr = LinearRegression() # XGBRegressor(gamma = 20, alpha = 10)
lr.fit(X_train, y_train)

LinearRegression()

In [481]:
r2_score(y_test, lr.predict(X_test)), r2_score(y_train, lr.predict(X_train))

(0.7455141528873459, 0.9848626069021182)

In [597]:
pseudo_y = torch.from_numpy(lr.predict(merged_target_genes.loc[:, perfect_features].values)).type(torch.float) #

In [598]:
from torch_geometric.nn import knn_graph
from torch_geometric.data import Data

edge_index = knn_graph(pseudo_y,k = 1, loop = False, num_workers = -1, cosine=False)

In [599]:
def index_to_mask(rows, index_array):
    mask_array = np.zeros(rows, dtype=int)
    mask_array[index_array] = 1
    return torch.from_numpy(mask_array.astype(np.bool_))

In [600]:
class GNN(torch.nn.Module):
    def __init__(self, input_dim, out_dim):
        super(GNN, self).__init__()
        self.conv_0 = GCNConv(input_dim, out_dim)
        # self.lin = Linear(input_dim, out_dim)

    def forward(self, x, edge_index):
        
        return self.conv_0(x, edge_index) #+ self.lin(x)

In [601]:
class Lin(torch.nn.Module):
    def __init__(self, input_dim, out_dim):
        super(Lin, self).__init__()
        self.lin = Linear(input_dim, out_dim)

    def forward(self, x, edge_index):
        
        return self.lin(x)

In [602]:
model = GNN(X_train.shape[-1], 1)
optim = torch.optim.Adam(params=model.parameters(), lr = 3e-3, weight_decay=1e-5)
loss_fn = torch.nn.MSELoss()
epochs = 50000

In [603]:
X = torch.from_numpy(merged_target_genes[perfect_features].astype(np.float32).values).type(torch.float)
y = torch.from_numpy(merged_target_genes["LN_IC50"].astype(np.float32).values).type(torch.float)
train_mask = index_to_mask(X.shape[0], train_idx)
test_mask = index_to_mask(X.shape[0], test_idx)


In [ ]:
for epoch in range(epochs):
    model.train()
    out = model(X, edge_index)
    loss = loss_fn(out[train_mask].squeeze(), y[train_mask])
    # print(loss.item())
    optim.zero_grad()
    loss.backward()
    optim.step()
    if epoch % 100 == 0:
        test_r2 = test(model, test_mask)
        train_r2 = test(model, train_mask)
        print(f"{epoch}: R2 Train: {train_r2:.4f}; R2 Test: {test_r2:.4f}; {loss.item():.4f}")

0: R2 Train: -1121391.0020; R2 Test: -1157965.4903; 4407462.0000
100: R2 Train: -5675.8827; R2 Test: -7294.5338; 19936.4902
200: R2 Train: -1659.4211; R2 Test: -2170.3112; 5824.7036
300: R2 Train: -686.5269; R2 Test: -945.3102; 2401.2520
400: R2 Train: -402.8813; R2 Test: -575.4248; 1407.3251
500: R2 Train: -272.2190; R2 Test: -399.9941; 951.2756
600: R2 Train: -194.0995; R2 Test: -293.7734; 679.0212
700: R2 Train: -143.0587; R2 Test: -223.7713; 501.2246
800: R2 Train: -108.7225; R2 Test: -176.5056; 381.6377
900: R2 Train: -85.3004; R2 Test: -144.2654; 300.0739
1000: R2 Train: -69.0338; R2 Test: -121.8636; 243.4411
1100: R2 Train: -57.4182; R2 Test: -105.7833; 203.0140
1200: R2 Train: -48.8246; R2 Test: -93.7329; 173.1158
1300: R2 Train: -42.2233; R2 Test: -84.2751; 150.1575
1400: R2 Train: -36.9740; R2 Test: -76.5308; 131.9071
1500: R2 Train: -32.6776; R2 Test: -69.9672; 116.9736
1600: R2 Train: -29.0806; R2 Test: -64.2590; 104.4734
1700: R2 Train: -26.0162; R2 Test: -59.2035; 93.8258

In [516]:
def test(model, mask):
    with torch.inference_mode():
        model.eval()
        out = model(X, edge_index)
        y_test = y[mask]
        y_pred = out[mask].squeeze()
    r2 = r2_score(y_test.numpy(), y_pred.numpy())
    return r2

## GNN 

In [426]:
def get_graph(data):
    global hierarchie_information_red
    data_hierarchy_merge = pd.merge(hierarchie_information_red, data, on = "SANGER_MODEL_ID", how = "right").drop_duplicates().reset_index(drop=True) # TODO WHY?
    tissue_index_list = data_hierarchy_merge.groupby("Tissue sub-type").apply(lambda g: g.index, include_groups=False).reset_index().rename(columns={0: "index_list"})
    tissue_based_connections = tissue_index_list["index_list"].apply(lambda index_list: list(product(index_list, index_list)))
    edge_index =  torch.from_numpy(np.concatenate(tissue_based_connections.values)).type(torch.long)
    x = torch.from_numpy(data_hierarchy_merge.loc[:, perfect_features].values.astype(np.float32)).type(torch.float)
    y = torch.from_numpy(data_hierarchy_merge.loc[:, "LN_IC50"].values.astype(np.float32)).type(torch.float)
    return Data(x = x, y = y, edge_index = edge_index.t())

In [427]:
device = torch.device("cuda:0")
train_graph = get_graph(train_data).to(device)
test_graph = get_graph(test_data).to(device)

In [16]:
test_graph

Data(x=[179, 388], edge_index=[2, 1447], y=[179])

In [17]:
class GNN(torch.nn.Module):
    def __init__(self, input_dim, out_dim):
        super(GNN, self).__init__()
        self.conv_0 = GATConv(input_dim, out_dim, dropout=0.4)
        self.lin_0 = Linear(input_dim, out_dim)

    def forward(self, x, edge_index):
        
        return self.conv_0(x, edge_index) + self.lin_0(x)

In [18]:
model = GNN(train_graph.x.shape[-1], 1).to(device)
optim = torch.optim.Adam(params= model.parameters(), lr = 1e-3)
loss_fn = torch.nn.MSELoss()

In [19]:
epochs = 10_000

In [20]:
for epoch in range(epochs):
    model.train()
    out = model(train_graph.x, train_graph.edge_index).squeeze()
    loss = loss_fn(out, train_graph.y)
    print(f"{loss.item():.4f}")
    optim.zero_grad()
    loss.backward()
    optim.step()

4923840.0000
4402887.5000
4234103.0000
3609756.7500
3249213.5000
2758667.5000
2269810.2500
1894413.3750
1655638.2500
1407719.6250
1095465.6250
932671.8750
716405.1875
586472.1250
473028.2188
326777.5312
244346.4688
180521.8750
126659.5312
89372.0391
73459.4766
59071.8164
60478.1484
57474.4883
66467.6094
79732.5781
93185.3125
103465.3281
114062.0234
123963.3906
133675.5781
137244.3125
140923.2500
139898.0625
139228.4375
136222.1250
129641.2500
124289.8828
114985.3203
106166.2969
92606.2344
88344.6328
77502.5625
70382.0391
63164.9922
54636.1172
47914.3281
43072.8633
37009.5156
33830.9766
30324.8652
29218.6738
27477.5605
24827.3301
24612.9941
23071.7930
25149.0625
26045.1230
26442.7676
26345.5703
25062.4238
25591.4473
25261.3281
24992.7891
23634.8652
25249.7383
24691.5918
22765.6191
20724.6660
23434.9219
20811.4688
20060.0273
19284.8945
20194.5547
17882.9180
16194.9814
16488.3613
15798.0518
16849.1816
16772.9961
15673.5186
14939.2910
15173.0840
15334.4570
14320.3779
14133.2666
14024.2627


In [21]:
def evaluate(graph, model):
    with torch.inference_mode():
        model.eval()
        out = model(graph.x, graph.edge_index)
    return r2_score(graph.y.cpu(), out.cpu())

In [22]:
evaluate(test_graph, model), evaluate(train_graph, model)

(-332.3985526216193, -1.0319367036141145)

In [29]:
import numpy as np

def get_quantiles(arr):
    return np.percentile(arr, [5, 25, 50, 75, 95])

In [30]:
result = get_quantiles(lr.coef_)
result

array([-5.12149535e-02, -4.02113410e-03,  5.34513321e-05,  4.63289591e-03,
        9.15903843e-02])

In [12]:
data_hierarchy_merge = pd.merge(hierarchie_information_red, train_data, on = "SANGER_MODEL_ID", how = "right").drop_duplicates().reset_index(drop=True)

In [39]:
data_hierarchy_merge["LN_IC50"].std()

1.8643455639613655

In [22]:
data_hierarchy_merge.groupby("Tissue sub-type")["LN_IC50"].apply(np.std).describe()

count    52.000000
mean      1.202418
std       0.809018
min       0.000000
25%       0.522743
50%       1.179105
75%       1.961861
max       2.924622
Name: LN_IC50, dtype: float64

In [32]:
store = dict()
for column in models.columns:
    if models[column].unique().shape[0] >1000:
        continue
    if models[column].unique().shape[0] < 50:
        continue
    # print(column)
    # print(models[column].describe())
    store[column] = models[column].unique().shape[0]
print(store)

{'parent_id': 91, 'synonyms': 485, 'cancer_type_ncit_id': 248, 'sample_site': 244, 'cancer_type_detail': 240, 'sampling_year': 54, 'doi': 189, 'pmed': 141, 'ploidy_wgs': 72, 'model_comments': 50, 'model_relations_comment': 238, 'HCMI': 102, 'age_at_sampling': 116, 'sample_treatment_details': 74}


In [16]:
models = models.rename(columns = {"model_id": "SANGER_MODEL_ID"})

In [17]:
data_model_merge = pd.merge(train_data, models, how = "left", on = "SANGER_MODEL_ID")

In [33]:
def target_std_describer_per_group(column_name):
    return data_model_merge.groupby(column_name)["LN_IC50"].apply(np.std).describe()

for column in store:
    print(column)
    print(target_std_describer_per_group(column))

parent_id
count    7.0
mean     0.0
std      0.0
min      0.0
25%      0.0
50%      0.0
75%      0.0
max      0.0
Name: LN_IC50, dtype: float64
synonyms
count    153.0
mean       0.0
std        0.0
min        0.0
25%        0.0
50%        0.0
75%        0.0
max        0.0
Name: LN_IC50, dtype: float64
cancer_type_ncit_id
count    136.000000
mean       0.723278
std        0.860547
min        0.000000
25%        0.000000
50%        0.445726
75%        1.266701
max        3.355576
Name: LN_IC50, dtype: float64
sample_site
count    97.000000
mean      0.607284
std       0.874782
min       0.000000
25%       0.000000
50%       0.000000
75%       1.417961
max       2.811982
Name: LN_IC50, dtype: float64
cancer_type_detail
count    136.000000
mean       0.723278
std        0.860547
min        0.000000
25%        0.000000
50%        0.445726
75%        1.266701
max        3.355576
Name: LN_IC50, dtype: float64
sampling_year
count    39.000000
mean      1.120683
std       0.874698
min       0.0

In [139]:
target_std_describer_per_group("tissue")

count    27.000000
mean      1.429012
std       0.857485
min       0.000000
25%       0.712971
50%       1.733462
75%       2.060139
max       2.924622
Name: LN_IC50, dtype: float64

In [413]:
models

,model_id,sample_id,patient_id,parent_id,model_name,synonyms,tissue,cancer_type,cancer_type_ncit_id,tissue_status,...,msh6_expression_by_ihc,braf_mutation_identified,braf_expression_by_ihc,pik3ca_mutation_identified,pten_expression_by_ihc,pten_mutation_identified,kras_mutation_identified,mismatch_repair_status,preoperative_ce_alevel,crispr_ko_data
0,SIDM01774,SIDS01659,SIDP01578,NaN,PK-59,NaN,Pancreas,Pancreatic Carcinoma,C3850,Metastasis,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
1,SIDM00192,SIDS00612,SIDP00541,NaN,SNU-1033,NaN,Large Intestine,Colorectal Carcinoma,C9383,Tumour,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
2,SIDM01447,SIDS01466,SIDP01347,NaN,SNU-466,NaN,Central Nervous System,Glioblastoma,C3058,Tumour,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
3,SIDM01554,SIDS01363,SIDP01247,NaN,IST-MES-2,NaN,Lung,Mesothelioma,C45662,Unknown,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
4,SIDM01689,SIDS01631,SIDP01557,NaN,MUTZ-5,NaN,Haematopoietic and Lymphoid,B-Lymphoblastic Leukemia,C8644,Tumour,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2150,SIDM02155,SIDS02010,SIDP01906,NaN,NZM3,NaN,Skin,Melanoma,C3224,Metastasis,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
2151,SIDM02157,SIDS02040,SIDP01935,NaN,WM3772F,NaN,Eye,Melanoma,C7712,Metastasis,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
2152,SIDM02159,SIDS02033,SIDP01928,SIDM02121,HAP1,NaN,Haematopoietic and Lymphoid,Chronic Myelogenous Leukemia,C3174,Tumour,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
2153,SIDM01339,SIDS01788,SIDP01690,SIDM01851,SLR24,NaN,Kidney,Kidney Carcinoma,C4033,Tumour,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True


In [430]:
data_model_merge.groupby("sample_site")["LN_IC50"].apply(np.unique)[:10] #.describe()

sample_site
Abdominal Mass                                 [4.929234, 5.055494]
Adrenal Cortex                                             [2.8067]
Adrenal Gland                        [6.066849, 6.233218, 6.382297]
Adrenal Tissue                                           [6.424382]
Alveolus                                                 [5.564495]
Ascites           [-0.737397, 1.090782, 1.573649, 2.049502, 2.19...
Ascites                                                  [9.829512]
Blood                                                    [5.839234]
Bone              [2.625352, 4.061616, 4.633708, 5.681226, 6.381...
Bone Marrow                                              [6.133551]
Name: LN_IC50, dtype: object

In [20]:
data_hierarchy_merge.columns

Index(['SANGER_MODEL_ID', 'Tissue sub-type', 'DATASET', 'NLME_RESULT_ID',
       'NLME_CURVE_ID', 'COSMIC_ID', 'CELL_LINE_NAME', 'TCGA_DESC', 'DRUG_ID',
       'DRUG_NAME',
       ...
       'TMEM134', 'ATP8B1', 'WNT4', 'ERV3-1', 'PSMD10P2', 'SCRN3', 'CDK10',
       'CTSV', 'RNA5SP440', 'EIF1AD'],
      dtype='object', length=408)

## Important note
GNNs are based on smoothing information i.e, we can only achieve an improvement with GNNs when the labels between neighbors are somewhat similar while the features values might have large variances (feature smoothing)
These large variances are then smoothed by the GNNs so that the similar labels within the group can be achieved (thats also why it doesnt make much sense to create the similarity based on all feature values -> use separate information for similarity construction or only a subset and then let the other features smoothen)

## Concat with drugs and Calculate similarity based on structural similarity of drugs -> similar drugs might have similar targets and similar reaction? But guess even liittle changes in structur can have high changes in IC50

In [157]:
networks = """N00001
N00229
N00021
N00015
N00215
N00019
N00217
N00233
N00216
N00001
N00001
N00001
N00229
N00229
N00229
N00021
N00021
N00021
N00015
N00015
N00015
N00215
N00215
N00215
N00019
N00019
N00019
N00217
N00217
N00217
N00233
N00233
N00233
N00216
N00216
N00216
N00033
N00231
N00039
N00045
N00037
N00234
N00043
N00220
N00218
N00033
N00033
N00033
N00231
N00231
N00231
N00039
N00039
N00039
N00045
N00045
N00045
N00037
N00037
N00037
N00234
N00234
N00234
N00043
N00043
N00043
N00220
N00220
N00220
N00218
N00218
N00218
N00096
N00103
N00096
N00096
N00096
N00103
N00103
N00103
N00056
N00056
N00056
N00056
N00086
N00086
N00086
N00086
N00062
N00062
N00062
N00062
N00063
N00063
N00063
N00063
N00053
N00094
N00219
N00053
N00053
N00053
N00094
N00094
N00094
N00219
N00219
N00219
N00026
N00028
N00227
N00026
N00026
N00026
N00028
N00028
N00028
N00227
N00227
N00227
N00079
N00079
N00079
N00079
N00243
N00243
N00243
N00243
N00083
N00083
N00083
N00083
N00066
N00069
N00090
N00091
N00066
N00066
N00066
N00069
N00069
N00069
N00090
N00090
N00090
N00091
N00091
N00091
N00098
N00101
N00098
N00098
N00098
N00101
N00101
N00101
N00239
N00239
N00239
N00239"""

In [175]:
cancer_networks = list(set(networks.split("\n")))

In [178]:
cancer_networks = list(map(lambda n: f"ne:{n}", cancer_networks))

In [167]:
hsa_to_network = pd.read_csv("hsa_to_network.txt", sep = "\t", header = None)

In [181]:
len(cancer_networks)

40

In [183]:
hsa_to_cancer = network = hsa_to_network[hsa_to_network[1].isin(cancer_networks)]

In [184]:
hsa_to_cancer

,0,1
71,hsa:1950,ne:N00033
72,hsa:1956,ne:N00033
73,hsa:5290,ne:N00033
74,hsa:5291,ne:N00033
75,hsa:5293,ne:N00033
...,...,...
2212,hsa:4258,ne:N00243
2213,hsa:4259,ne:N00243
2214,hsa:653689,ne:N00243
2215,hsa:9446,ne:N00243


In [208]:
def get_gene_idx_to_gene_symbol(human_genome):
    sem_split = human_genome[3].str.split("; ")
    comma_split = sem_split.map(lambda n: n[0]).str.split(",")
    exploded_gene_symbols = comma_split.explode().str.strip().to_frame()
    exploded_gene_symbols = exploded_gene_symbols.rename(columns={3:"symbol"})
    exploded_gene_symbols = exploded_gene_symbols.drop_duplicates(subset = "symbol")
    exploded_gene_symbols["human_genome_index"] = exploded_gene_symbols.index
    exploded_gene_symbols = exploded_gene_symbols.reset_index(drop=True)
    return exploded_gene_symbols

human_genome = pd.read_csv("human_genome.tsv", sep="\t", header=None) ## merging via gene symbol
human_genome["human_genome_index"] = human_genome.index
gene_idx_to_gene_symbol_df = get_gene_idx_to_gene_symbol(human_genome)
symbol_to_hsa = pd.merge(gene_idx_to_gene_symbol_df, human_genome, on = "human_genome_index").loc[:, ["symbol", 0]]

In [211]:
symbol_to_hsa = symbol_to_hsa.rename(columns= {0:"hsa"})
hsa_to_cancer = hsa_to_cancer.rename(columns= {0:"hsa"})

In [225]:
network_to_symbol = pd.merge(hsa_to_cancer, symbol_to_hsa, on = "hsa", how = "left")

In [222]:
model_expression_data_tp = model_expression_data.T.reset_index()
model_expression_data_tp.columns = model_expression_data_tp.iloc[0]
model_expression_data_tp = model_expression_data_tp.drop(model_expression_data_tp.index[0]).reset_index(drop=True)
# model_expression_data_tp = model_expression_data_tp.set_index(model_expression_data_tp.columns[0])

In [224]:
model_expression_data_tp = model_expression_data_tp.rename(columns={"SANGER_MODEL_ID": "symbol"})

In [229]:
cancer_gene_networks = pd.merge(network_to_symbol, model_expression_data_tp, on = "symbol", how = "inner")

In [243]:
cancer_gene_networks = cancer_gene_networks.drop(["hsa", "symbol"], axis= 1)

In [334]:
cancer_gene_networks

,1,SIDM00001,SIDM00002,SIDM00003,SIDM00005,SIDM00006,SIDM00007,SIDM00008,SIDM00009,SIDM00011,...,SIDM02071,SIDM02072,SIDM02073,SIDM02074,SIDM02075,SIDM02076,SIDM02077,SIDM02078,SIDM02079,SIDM02080
0,ne:N00033,0.000000,1.639648,1.250000,0.340088,2.119141,0.419922,0.090027,0.029999,5.199219,...,0.000000,0.700195,0.059998,0.010002,0.360107,0.020004,0.090027,0.049988,0.010002,0.180054
1,ne:N00033,0.070007,1.809570,6.199219,0.229980,39.031250,29.437500,92.875000,0.520020,0.130005,...,124.437500,7.941406,9.351562,14.726562,16.656250,10.140625,8.570312,14.281250,9.343750,17.343750
2,ne:N00033,14.976562,18.703125,26.656250,14.289062,18.734375,11.671875,7.191406,29.062500,9.078125,...,8.070312,3.539062,5.011719,5.628906,7.171875,1.730469,8.312500,5.289062,3.230469,12.007812
3,ne:N00033,9.117188,8.570312,43.687500,18.515625,11.101562,15.250000,13.976562,14.898438,31.890625,...,30.234375,15.156250,15.843750,49.031250,38.500000,6.929688,22.265625,33.218750,16.843750,48.312500
4,ne:N00033,116.187500,3.269531,26.890625,0.600098,7.980469,34.062500,1.120117,242.500000,62.406250,...,11.218750,0.479980,1.280273,1.059570,2.810547,0.330078,11.226562,1.269531,0.449951,0.629883
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
583,ne:N00243,0.970215,5.230469,4.921875,37.843750,22.015625,6.761719,14.640625,6.718750,17.906250,...,81.750000,103.187500,83.687500,44.875000,71.000000,34.250000,36.562500,108.562500,17.859375,47.500000
584,ne:N00243,2.429688,17.984375,45.781250,27.406250,42.625000,27.031250,5.839844,15.140625,43.375000,...,28.015625,98.562500,79.437500,83.500000,85.000000,222.375000,68.750000,146.500000,62.531250,67.125000
585,ne:N00243,0.070007,8.210938,15.632812,117.750000,0.070007,15.570312,4.980469,0.209961,19.515625,...,0.000000,1.129883,5.179688,0.080017,0.340088,0.360107,1.719727,0.140015,3.589844,0.689941
586,ne:N00243,107.375000,125.687500,471.000000,140.500000,122.625000,229.000000,191.125000,163.250000,510.750000,...,200.375000,199.125000,208.750000,108.500000,231.250000,536.000000,183.625000,117.875000,85.937500,239.500000


In [262]:
for column in cancer_gene_networks.columns:
    if column == 1:
        continue
    cancer_gene_networks[column] = cancer_gene_networks[column].astype("float16")

In [268]:
cancer_gene_networks_aggr = cancer_gene_networks.groupby(1).apply(np.sum, axis = 0)

/tmp/ipykernel_626781/2547204752.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  cancer_gene_networks_aggr = cancer_gene_networks.groupby(1).apply(np.sum, axis = 0)


In [277]:
cancer_gene_networks_aggr_transp = cancer_gene_networks_aggr.transpose().iloc[1:, :].reset_index().rename(columns={"index": "SANGER_MODEL_ID"})

In [284]:
cancer_gene_networks_aggr_transp_merge = pd.merge(cancer_gene_networks_aggr_transp, drug_targets.loc[:, ["LN_IC50", "SANGER_MODEL_ID"]], on  ="SANGER_MODEL_ID", how = "inner")

In [287]:
cancer_gene_networks_aggr_transp_merge = cancer_gene_networks_aggr_transp_merge.iloc[:, 1:]

In [291]:
train_data, test_data = train_test_split(cancer_gene_networks_aggr_transp_merge, test_size = 0.25)

In [301]:
X_train = train_data.iloc[:, :-1].values
X_test = test_data.iloc[:, :-1].values
y_train = train_data.iloc[:, -1].values
y_test = test_data.iloc[:,-1].values

In [318]:
lr = XGBRegressor(alpha = 5, gamma = 5)
lr.fit(X_train, y_train)

XGBRegressor(alpha=5, base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=5, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, ...)

In [319]:
r2_score(y_test, lr.predict(X_test)), r2_score(y_train, lr.predict(X_train))

(0.12997851459662002, 0.6092009558612513)

In [320]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, lr.predict(X_test)), mean_squared_error(y_train, lr.predict(X_train))

(3.112503323289239, 1.3292704352348699)

In [340]:
cancer_gene_networks = cancer_gene_networks.iloc[:, 1:].transpose().reset_index().rename(columns={"index": "SANGER_MODEL_ID"})

In [342]:
cancer_gene_networks_merge = pd.merge(cancer_gene_networks, drug_targets.loc[:, ["LN_IC50", "SANGER_MODEL_ID"]], on  ="SANGER_MODEL_ID", how = "inner")

In [385]:
train_data, test_data = train_test_split(cancer_gene_networks_merge, test_size = 0.25)

X_train = train_data.iloc[:, 1:-1].values.astype(np.float32)
X_test = test_data.iloc[:, 1:-1].values.astype(np.float32)
y_train = train_data.iloc[:, -1].values.astype(np.float32)
y_test = test_data.iloc[:,-1].values.astype(np.float32)

In [386]:
np.isnan(y_test).sum()

0

In [411]:
lr = XGBRegressor(alpha = 50)
lr.fit(X_train, y_train)

XGBRegressor(alpha=50, base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, ...)

In [412]:
r2_score(y_test, lr.predict(X_test)), r2_score(y_train, lr.predict(X_train))

(0.2675618658733636, 0.5919725618789169)